# Using the VidigiStore to Simplify Model Code Changes

In vidigi 0.0.5, a new class `VidigiStore` has been added that can be used almost exactly like a resource - reducing the amount of rewriting required to incorporate vidigi into your model. 

This allows us to use the pattern

```python
with self.treatment_cubicles.request() as req:

    treatment_resource = yield req

    ### Continue all code in the indented portion that requires the resource, with the resource
    ### automatically being returned to the store when the indented portion completes
```

Instead of 

```python
treatment_resource = yield self.treatment_cubicles.get()

### Continue all code that requires the resource

self.treatment_cubicles.put(treatment_resource)
```

(even though we are still using a Store behind the scenes)


This minimizes the syntax changes and rewriting that are required when converting an existing model built using resources to a vidigi-compatible state.

When setting up the resources, we simply use the pattern

```
from vidigi.resources import VidigiStore

...

self.treatment_cubicles = VidigiStore(
    self.env,
    num_resources=g.n_cubicles, # or wherever you are storing your resource counts
    )
```

In [2]:
from examples.example_6_simplest_case_storewrapper.ex_6_model_classes_with_vidigi_logging import Trial, g
from vidigi.animation import animate_activity_log
import pandas as pd
import plotly.io as pio
pio.renderers.default = "notebook"
import os

In [3]:
#| echo: false
#| output: asis
# Path to the external Python script
file_path = "ex_6_model_classes_with_vidigi_logging.py"

# Read the file content
if os.path.exists(file_path):
    with open(file_path, "r") as f:
        code_content = f.read()
else:
    code_content = "File not found."
with open(file_path, "r") as f:
    code_content = f.read()

# Print the Quarto `{details}` block for collapsible output
print(f"""
:::{{.callout-note collapse="true"}}
### View Imported Code, which has had logging steps added at the appropriate points in the 'model' class

```python
{code_content}
```

:::

""")


:::{.callout-note collapse="true"}
### View Imported Code, which has had logging steps added at the appropriate points in the 'model' class

```python
import random
import numpy as np
import pandas as pd
import simpy
from sim_tools.distributions import Exponential, Lognormal
from vidigi.resources import VidigiStore


# Class to store global parameter values.  We don't create an instance of this
# class - we just refer to the class blueprint itself to access the numbers
# inside.
class g:
    '''
    Create a scenario to parameterise the simulation model

    Parameters:
    -----------
    random_number_set: int, optional (default=DEFAULT_RNG_SET)
        Set to control the initial seeds of each stream of pseudo
        random numbers used in the model.

    n_cubicles: int
        The number of treatment cubicles

    trauma_treat_mean: float
        Mean of the trauma cubicle treatment distribution (Lognormal)

    trauma_treat_var: float
        Variance of the trauma cubicle treat

In [4]:
my_trial = Trial()

my_trial.run_trial()

4 nurses



In [5]:
my_trial.all_event_logs.head(50)

,patient,pathway,event_type,event,time,resource_id,run
0,1,Simplest,arrival_departure,arrival,0.000000,NaN,0
1,1,Simplest,queue,treatment_wait_begins,0.000000,NaN,0
2,1,Simplest,resource_use,treatment_begins,0.000000,1.0,0
3,2,Simplest,arrival_departure,arrival,3.399660,NaN,0
4,2,Simplest,queue,treatment_wait_begins,3.399660,NaN,0
5,2,Simplest,resource_use,treatment_begins,3.399660,2.0,0
6,3,Simplest,arrival_departure,arrival,8.497645,NaN,0
7,3,Simplest,queue,treatment_wait_begins,8.497645,NaN,0
8,3,Simplest,resource_use,treatment_begins,8.497645,3.0,0
9,4,Simplest,arrival_departure,arrival,8.596678,NaN,0


In [6]:
event_position_df = pd.DataFrame([
                    {'event': 'arrival',
                     'x':  50, 'y': 300,
                     'label': "Arrival" },

                    # Triage - minor and trauma
                    {'event': 'treatment_wait_begins',
                     'x':  205, 'y': 275,
                     'label': "Waiting for Treatment"},

                    {'event': 'treatment_begins',
                     'x':  205, 'y': 175,
                     'resource':'n_cubicles',
                     'label': "Being Treated"},

                    {'event': 'depart',
                     'x':  270, 'y': 70,
                     'label': "Exit"}

                ])


In [9]:
animate_activity_log(
        event_log=my_trial.all_event_logs[my_trial.all_event_logs['run']==1],
        event_position_df= event_position_df,
        scenario=g(),
        entity_col_name="patient",
        debug_mode=True,
        setup_mode=False,
        every_x_time_units=1,
        include_play_button=True,
        entity_icon_size=20,
        text_size=20,
        gap_between_entities=6,
        gap_between_queue_rows=25,
        plotly_height=700,
        frame_duration=200,
        plotly_width=1200,
        override_x_max=300,
        override_y_max=500,
        limit_duration=g.sim_duration,
        wrap_queues_at=25,
        step_snapshot_max=125,
        time_display_units="dhm",
        display_stage_labels=False,
        add_background_image="https://raw.githubusercontent.com/Bergam0t/vidigi/refs/heads/main/examples/example_1_simplest_case/Simplest%20Model%20Background%20Image%20-%20Horizontal%20Layout.drawio.png",
    )

Animation function called at 12:36:45
Iteration through time-unit-by-time-unit logs complete 12:36:48
Snapshot df concatenation complete at 12:36:48
Reshaped animation dataframe finished construction at 12:36:48
Placement dataframe finished construction at 12:36:48
Output animation generation complete at 12:36:51
Total Time Elapsed: 5.62 seconds
